In [ ]:
import os, sys
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))

import hpc05
import holoviews as hv

import numpy as np
import kwant
import scipy.constants
import cmath
import functools as ft


import sns_system, spectrum, topology, supercurrent

from IPython.core.magic import register_cell_magic
import ipywidgets as widgets
run_cluster = widgets.Checkbox(
    value=False,
    description='Check to run cluster',
    disabled=False
)

interactive_shell = get_ipython()
@register_cell_magic
def cluster_mark_checked(line, cell):
    if run_cluster.value is True:
        run_cluster.value = False
        interactive_shell.run_cell(cell)
    else:
        return "Command not run. Check the box above to run."

del(cluster_mark_checked)
run_cluster

In [ ]:
%%cluster_mark_checked
# client, dview, lview = hpc05.connect_ipcluster(20, folder='two_dim_majoranas')
hpc05.kill_remote_ipcluster()
# client, dview, lview = hpc05.start_remote_and_connect(20, folder='two_dim_majoranas')

In [ ]:
import numpy as np
import scipy.constants
import cmath

import functools as ft

import sns_system, spectrum, topology, supercurrent

In [ ]:
constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

# Make T shaped system

In [ ]:

def l_shaped_system(a, L_m, L_x, W_up, W_down, l_on,
                    transverse_soi=True,
                    mu_from_bottom_of_spin_orbit_bands=True,
                    k_x_in_sc=False, leaded=True, **_):

    #     HAMILTONIAN DEFINITIONS
    template_strings = sns_system.get_template_strings(
        transverse_soi, mu_from_bottom_of_spin_orbit_bands, k_x_in_sc)
    
    conservation_matrix = -supercurrent.sigz
    
    # TURN HAMILTONIAN STRINGS INTO TEMPLATES
    kwargs = dict(coords=('x', 'y'), grid_spacing=a)
    template_barrier = kwant.continuum.discretize(
        template_strings['ham_barrier'], **kwargs)
    template_normal = kwant.continuum.discretize(
        template_strings['ham_normal'], **kwargs)
    template_sc_left = kwant.continuum.discretize(
        template_strings['ham_sc_left'], **kwargs)
    template_sc_right = kwant.continuum.discretize(
        template_strings['ham_sc_right'], **kwargs)

    def union_shape(shapes):
        def _shape(pos):
            res = False
            for shape in shapes:
                res |= shape(pos)
            return res
        return _shape
        
    def intersection_shape(shape_A, shape_B):
        def _shape(pos):
            return shape_A(pos) and not shape_B(pos)
        return _shape
    

    def middle_shape(site):
        x, y = site.pos
        return 0 <= x < L_x and 0 <= y < L_m
    
    def top_shape_block(site):
        x, y = site.pos
        return -W_up*l_on <= x < L_x and (1-l_on)*L_m//4+ L_m//4 <= y < L_m + W_up
    top_shape = intersection_shape(top_shape_block, middle_shape)
    
    def down_shape_block(site):
        x, y = site.pos
        return 0 <= x < L_x + W_down*l_on and -W_down <= y < l_on*3*L_m//4
    down_shape = intersection_shape(down_shape_block, middle_shape)
    
    # BUILD FINITE SYSTEM
    syst = kwant.Builder()
    syst.fill(template_normal, middle_shape, (0, a))
#     syst.fill(template_barrier, shape_barrier, (0, 0)[::-1])
#     syst.fill(template_barrier, shape_barrier, (L_m, 0)[::-1])
    
    syst.fill(template_sc_left, top_shape, (a, L_m))
    syst.fill(template_sc_right, down_shape, (a, -a))
    
    syst = syst.finalized()

    return syst

In [ ]:
syst_pars = dict(a=5, l_on=1, L_m=2000, L_x=100, W_up=200, W_down=200,
                 offset_lead=100, k_x_in_sc=True, leaded=False)

params_raw= dict(g_factor_middle = 26,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 5,
                 alpha_middle = 20,
                 alpha_left = 0,
                 alpha_right = 0,
                 Delta_left = 2,
                 Delta_right = 2,
                 B = 1.2,
                 phase = np.pi,
                 T = 0.0,
                 V = 0.0)

params = dict(**constants,
              **params_raw)

In [ ]:
import formulas as f
vf = f.fermi_velocity(4, 0.02)
f.thouless_energy(vf, 400/1.4)/26/params['mu_B']


In [ ]:
import plotting_results
syst = l_shaped_system(**syst_pars)
ax1=plotting_results.plot_syst(syst=syst, params=params);
ax1.delaxes(ax1.axes[1])
ax1.set_size_inches(20, 10)
ax1.savefig('L_system')

In [ ]:
hv.extension('matplotlib')

In [ ]:
%%output filename='./first_and_second_energy_table' fig='png'
distributed_sns.ptable(params_raw) + distributed_sns.ptable(syst_pars)

In [ ]:
def lowest_two(pB):    
    _params = params.copy()
    _params['phase'] = pB[0]
    _params['B'] = pB[1]
    
    ham = syst.hamiltonian_submatrix(params=_params, sparse=True)
    d = kwant.operator.Density(syst)
    e, ev = spectrum.sparse_diag(ham,4,0)
    return np.sort(e)[2:4]


In [ ]:
import adaptive
adaptive.notebook_extension()

In [ ]:
learner = adaptive.Learner2D(lowest_two, [[0,2*np.pi], [-4,4]])

In [ ]:
runner = adaptive.Runner(learner)
runner.live_info()

In [ ]:
runner.cancel()

In [ ]:
hv.extension('matplotlib')

In [ ]:
_=learner.plot(800)

In [ ]:
a=_.items()[0][1].Image.I

In [ ]:
hv.operation.contours(a, levels=10)

In [ ]:
%%opts Image [colorbar=True]
# %%output filename='./first_and_second_energy_square' fig='png'
hv.HoloMap(learner.plot(800).redim(x='phase', y='B'), kdims='nth lowest eigenvalue')

In [ ]:
ham = syst.hamiltonian_submatrix(params=params, sparse=True)
d = kwant.operator.Density(syst)
e, ev = spectrum.sparse_diag(ham,4,0)

In [ ]:
_ev = np.abs(d(ev[:,0]))

In [ ]:
e

In [ ]:
import matplotlib.pyplot as plt
ax=plt.subplot(111)
kwant.plotter.map(syst, _ev, ax=ax);
ax1=plotting_results.plot_syst(syst=syst, params=params, ax=ax);


In [ ]:
ax.savefig('snake_major')

In [ ]:
from multiprocessing import Pool

In [ ]:
p=Pool(12)

In [ ]:
def G(e):
    smat = kwant.smatrix(syst, energy=e, params=params)
    return smat.conductance_matrix()[0][0]

In [ ]:
G(.11)

In [ ]:
res = p.map(G, np.linspace(-.2, .2, 101))